# Multi-CURIE Query wrapper for Monarch SemSimian

In [1]:
import requests
import json

## Local 'development' MMCQ query endpoint

In [2]:
MMCQ = "http://localhost:8080/1.5"

## Monarch Multi-CURIE Query: "Meta Knowledge Graph"

In [3]:
MMCQ_MKG = f"{MMCQ}/meta_knowledge_graph"

In [4]:
response = requests.get( MMCQ_MKG )
print(json.dumps(response.json(), indent=4)) if response.status_code == 200 else print(f"MMCQ not accessible, HTTP code == {response.status_code}")

{
    "nodes": {
        "biolink:PhenotypicFeature": {
            "id_prefixes": [
                "HP"
            ]
        },
        "biolink:Disease": {
            "id_prefixes": [
                "MONDO"
            ]
        }
    },
    "edges": [
        {
            "subject": "biolink:PhenotypicFeature",
            "predicate": "biolink:has_member",
            "object": "biolink:PhenotypicFeature",
            "association": "biolink:PhenotypicFeatureToPhenotypicFeatureAssociation"
        },
        {
            "subject": "biolink:Disease",
            "predicate": "biolink:has_phenotype",
            "object": "biolink:PhenotypicFeature",
            "association": "biolink:DiseaseToPhenotypicFeatureAssociation"
        },
        {
            "subject": "biolink:PhenotypicFeature",
            "predicate": "biolink:similar_to",
            "object": "biolink:PhenotypicFeature",
            "association": "biolink:PhenotypicFeatureToPhenotypicFeatureAssociation"
 

In [5]:
MMCQ_QUERY = f"{MMCQ}/query"

In [6]:
def generate_trapi_query(input_node_type,output_node_type,input_curies,predicate,input_is_subject=True):
    envelope = {"message":{"query_graph": {"nodes":{"input":{},"output":{}}, "edges":{"edge_0":{}}}}}
    input_node = envelope["message"]["query_graph"]["nodes"]["input"]
    input_node["categories"] = [input_node_type]
    input_node["is_set"] = True
    input_node["set_interpretation"]="MANY"
    input_node["ids"] = ["uuid:1"]
    input_node["member_ids"] = input_curies
    output_node = envelope["message"]["query_graph"]["nodes"]["output"]
    output_node["categories"] = [output_node_type]
    query_edge = envelope["message"]["query_graph"]["edges"]["edge_0"]
    if input_is_subject:
        query_edge["subject"] = "input"
        query_edge["object"] = "output"
    else:
        query_edge["subject"] = "output"
        query_edge["object"] = "input"
    query_edge["predicates"]  = [ predicate ]
    #query_edge["knowledge_type"] = "inferred"
    return envelope

In [7]:
def phenotypes_to_disease(phenotypes):
    #Phenotype to Disease
    input_type = "biolink:PhenotypicFeature"
    output_type = "biolink:Disease"
    predicate = "biolink:has_phenotype"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, phenotypes, predicate, input_is_subject)
    p2d_response = requests.post( MMCQ_QUERY, json = query )
    print(p2d_response.status_code)
    return p2d_response.json()

In [8]:
# The phenotypes-to-geno use case isn't yet supported by the MMCQ (but could be sometime soon?) 
# 
# def phenotypes_to_gene(phenotypes):
#     #Phenotype to Gene
#     input_type = "biolink:PhenotypicFeature"
#     output_type = "biolink:Gene"
#     predicate = "biolink:genetically_associated_with"
#     input_is_subject = "True"
#     query = generate_trapi_query( input_type, output_type, phenotypes, predicate, input_is_subject)
#     response = requests.post( MMCQ_QUERY, json = query )
#     print(response.status_code)
#     return response.json()

In [9]:
def print_results(resp):
    for result in resp["message"]["results"]:
        nb = result["node_bindings"]["output"][0]["id"]
        name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
        print(nb, name) 

## Sample query: Leigh Disease

In [10]:
leigh_phenotypes = ["HP:0000739", #| Anxiety, 
                    "HP:0001288", # | Gait disturbance, 
                    "HP:0001252", # | Hypotonia, 
                    "HP:0001250", # | Seizure, 
                    "HP:0000750", # | Delayed speech and language development, 
                    "HP:0002378", # | Hand tremor, 
                    "HP:0002019", # | Constipation, 
                    "HP:0007146"] # | Bilateral basal ganglia lesions

In [11]:
response = phenotypes_to_disease(leigh_phenotypes)

200


### Raw TRAPI response

In [12]:
print(json.dumps(response, indent=4))

{
    "message": {
        "query_graph": {
            "nodes": {
                "input": {
                    "ids": [
                        "uuid:1"
                    ],
                    "categories": [
                        "biolink:PhenotypicFeature"
                    ],
                    "set_interpretation": "MANY",
                    "constraints": [],
                    "member_ids": [
                        "HP:0000739",
                        "HP:0001288",
                        "HP:0001252",
                        "HP:0001250",
                        "HP:0000750",
                        "HP:0002378",
                        "HP:0002019",
                        "HP:0007146"
                    ],
                    "is_set": true
                },
                "output": {
                    "ids": null,
                    "categories": [
                        "biolink:Disease"
                    ],
                    "set_interpretation": "

### List of Diseases Matched

In [13]:
print_results(response)

MONDO:0007496 dystonia 12
MONDO:0044637 infantile-onset generalized dyskinesia with orofacial involvement
MONDO:0957203 intellectual developmental disorder, X-linked 111
MONDO:0008590 tremor, hereditary essential, 1
MONDO:0010313 intellectual disability, X-linked 63
MONDO:0859257 intellectual developmental disorder with language impairment and early-onset DOPA-responsive dystonia-parkinsonism
MONDO:0013708 intellectual disability, autosomal recessive 25
MONDO:0859211 neurodevelopmental disorder with hyperkinetic movements and dyskinesia
MONDO:0020762 diencephalic-mesencephalic junction dysplasia syndrome 2
MONDO:0020846 intellectual disability, autosomal recessive 64


In [14]:
# See above, 'phenotypes to gene' matching is not yet implemented in MMCQ
# response = phenotypes_to_gene(leigh_phenotypes)
# print_results(response)

## Sotos-2 (Malan syndrome)

In [15]:
sotos_2_phenotypes = [
    "HP:0000718", # aggression         | 2 yrs - present
    "HP:0000739", #anxiety                | 2 yrs - present
    "HP:0002616", # Aortic root aneurysm HP:0002616 | 10 yrs - present
    "HP:0002019", #constipation (occasional) HP:0002019 |        birth - present
    #occasional bright lime-green stool | 5 yrs - present
    "HP:0000678", #crowded teeth        HP:0000678         
    "HP:0000886", # Deformed rib cage HP:0000886 | 6 months - present
    "HP:0002046", #heat intolerance        HP:0002046 
    "HP:0002705", #high arched palate        HP:0002705 
    "HP:0001290", #hypotonia        HP:0001290 |        birth - present
    "HP:0000737", # irritable        HP:0000737 |        2 yrs - present
    "HP:0001382", # Joint hypermobility HP:0001382 | birth - present
    "HP:0000720", # mood swings        HP:0000720 |        2 yrs - present
    "HP:0012801", # narrow jaw        HP:0012801
    "HP:0010865", # Oppositional defiant disorder HP:0010865 | 2 yrs - present
    #Destructive tendencies | 3 yrs - present
    "HP:0002172", # Postural instability HP:0002172 | birth - present
    "HP:0001250", # seizures        HP:0001250 |        7 yrs - present
    "HP:0001649", # Tachycardia HP:0001649 | 1 yr - present
    "HP:0001337", # tremor        HP:0001337 | in hands 11 yrs - present
    #Vascular skin abnormality (superficial blood vessels that rupture easily) HP:0011276 |  birth - present
    "HP:0001123" # Visual field defect HP:0001123 | birth - present
    # Seasonal allergies | birth - present
    #Food allergy | 4 yrs - present
]

In [16]:
response = phenotypes_to_disease(sotos_2_phenotypes)
print_results(response)

200
MONDO:0017782 developmental and speech delay due to SOX5 deficiency
MONDO:0018670 symptomatic form of fragile X syndrome in female carrier
MONDO:0014950 aortic aneurysm, familial thoracic 10
MONDO:0030051 intellectual developmental disorder with autistic features and language delay, with or without seizures
MONDO:0007496 dystonia 12
MONDO:0957576 parkinson disease 25, autosomal recessive early-onset, with impaired intellectual development
MONDO:0017279 young-onset Parkinson disease
MONDO:0018546 serotonin syndrome
MONDO:0016305 atypical pantothenate kinase-associated neurodegeneration
MONDO:0032698 neurodevelopmental disorder with central and peripheral motor dysfunction


In [17]:
# response = phenotypes_to_gene(phenotypes_2)
# print_results(response)

## TBCD

In [18]:
tbcd_phenotypes=[
    "HP:0002098", # | Respiratory distress, 
    "HP:0001252", # | Hypotonia, 
    "HP:0001250" # | Seizure
]

In [19]:
response = phenotypes_to_disease(tbcd_phenotypes)
print_results(response)

200
MONDO:0017049 obsolete hypomyelination neuropathy-arthrogryposis syndrome
MONDO:0024508 epilepsy, hot water, 1
MONDO:0014582 congenital myasthenic syndrome 2C
MONDO:0017856 X-linked spasticity-intellectual disability-epilepsy syndrome
MONDO:0007147 obstructive sleep apnea syndrome
MONDO:0008845 atonic-astatic syndrome of Foerster
MONDO:0008027 muscular atrophy, malignant neurogenic
MONDO:0859263 developmental delay, impaired speech, and behavioral abnormalities, with or without seizures
MONDO:0009186 epilepsy, photogenic, with spastic diplegia and intellectual disability
MONDO:0010451 intellectual disability, X-linked 41


In [20]:
# response = phenotypes_to_gene(tbcd_phenotypes)
# print_results(response)

## WDFY3 mutations are associated with MICROCEPHALY 18, PRIMARY, AUTOSOMAL DOMINANT; MCPH18 (MIM 617520)

In [21]:
wdyf_phenotypes = [
    #Global developmental delay 
    "HP:0001263",
    #Seizure 
    "HP:0001250",
    #Neurodevelopmental delay 
    "HP:0012758",
    #Delayed social development 
    "HP:0012434"
]

In [22]:
response = phenotypes_to_disease(wdyf_phenotypes)
print_results(response)

200
MONDO:0010450 intellectual disability, X-linked 89
MONDO:0007366 seizures, benign familial neonatal, 2
MONDO:0013705 intellectual disability, autosomal recessive 19
MONDO:0030958 dystonia 35, childhood-onset
MONDO:0010413 intellectual disability, X-linked 95
MONDO:0013706 intellectual disability, autosomal recessive 23
MONDO:0010454 intellectual disability, X-linked 88
MONDO:0014815 intellectual disability, autosomal recessive 52
MONDO:0859261 attention deficit-hyperactivity disorder 8
MONDO:0012618 intellectual disability, autosomal recessive 10


In [23]:
# response = phenotypes_to_gene(wdyf_phenotypes)
# print_results(response)

## Recurrent fever AND 2 Gene Mutations (AFG3L2 and TNFRSF13B)

In [24]:
twogene_phenotypes = [
    #Recurrent fever 
    "HP:0001954",
    #hip dysplasia        
    "HP:0001385",
    #joint hypermobility        
    "HP:0001382",
    #optic disc pallor        
    "HP:0000543",        
    #recurrent infections        
    "HP:0002719",        
    #thin corpus callosum        
    "HP:0002079",
    #Global developmental delay 
    "HP:0001263",
    #Abnormality of vision 
    "HP:0000504",
    #Nystagmus 
    "HP:0000639",
    #Drooling 
    "HP:0002307",
    #Dysphagia 
    "HP:0002015",
    #Hypotonia 
    "HP:0001252"
    #Eczema 
    "HP:0000964",
    #Inability to walk 
    "HP:0002540",
    #Septo-optic dysplasia 
    "HP:0100842"
    #Postural instability 
    "HP:0002172"
]

In [25]:
response = phenotypes_to_disease(twogene_phenotypes)
print_results(response)

200
MONDO:0859313 neurodevelopmental disorder with speech impairment and with or without seizures
MONDO:0011897 leukoencephalopathy-ataxia-hypodontia-hypomyelination syndrome
MONDO:0020777 congenital disorder of glycosylation with defective fucosylation 2
MONDO:0013165 hereditary spastic paraplegia 45
MONDO:0014979 myoclonus, intractable, neonatal
MONDO:0859250 neurodevelopmental disorder with microcephaly, hypotonia, nystagmus, and seizures
MONDO:0054782 leukodystrophy, hypomyelinating, 15
MONDO:0054791 leukodystrophy, hypomyelinating, 16
MONDO:0859221 Yoon-Bellen neurodevelopmental syndrome
MONDO:0013722 hypomyelinating leukodystrophy 8 with or without oligodontia and-or hypogonadotropic hypogonadism


In [26]:
# response = phenotypes_to_gene(twogene_phenotypes)
# print_results(response)

## Autism, SON-related disorder

In [27]:
autism_phenotypes = [
    # Autistic behavior
    "HP:0000729",
    # Neurodevelopmental delay
    "HP:0012758",
    # Intellectual disability
    "HP:0001249",
    # Ventricular septal defect
    "HP:0001629",
    # Abnormal facial shape
    "HP:0001999",
    # high arched palate
    "HP:0002705",
    # high nasal bridge
    "HP:0000426",
    # shallow orbits
    "HP:0000586",
    # Abnormality of the palmar creases
    "HP:0010490"
]

In [28]:
response = phenotypes_to_disease(autism_phenotypes)
print_results(response)

200
MONDO:0010344 intellectual disability, X-linked 45
MONDO:0010398 syndromic X-linked intellectual disability 14
MONDO:0030674 Teebi hypertelorism syndrome 2
MONDO:0011538 frontoocular syndrome
MONDO:0032697 Houge-Janssens syndrome 3
MONDO:0019767 hamel cerebro-palato-cardiac syndrome
MONDO:0010039 congenital heart defect-round face-developmental delay syndrome
MONDO:0013528 intellectual disability, autosomal recessive 14
MONDO:0010194 Weill-Marchesani syndrome 1
MONDO:0032786 Noonan syndrome 11


## MED12 and SPG7

In [29]:
med12_spg7_phenotypes = [
    "HP:0033454", # Tube feeding,
    "HP:0001601", # Laryngomalacia,
    "HP:0001263", # Global developmental delay,
    "HP:0000508", # Ptosis,
    "HP:0001770", # Toe syndactyly,
    "HP:0000365", # Hearing impairment,
    "HP:0001999", # Abnormal facial shape,
    "HP:0012448", # Delayed myelination,
    "HP:0033725"  # Thin corpus callosum
]

In [30]:
response = phenotypes_to_disease(med12_spg7_phenotypes)
print_results(response)

200
MONDO:0859313 neurodevelopmental disorder with speech impairment and with or without seizures
MONDO:0012623 intellectual disability, autosomal recessive 4
MONDO:0013858 pontine tegmental cap dysplasia
MONDO:0859266 neurodevelopmental disorder with severe motor impairment, absent language, cerebral hypomyelination, and brain atrophy
MONDO:0012368 aminoacylase 1 deficiency
MONDO:0032596 myasthenic syndrome, congenital, 23, presynaptic
MONDO:0033667 Delpire-McNeill syndrome
MONDO:0033311 Joubert syndrome 33
MONDO:0014664 Joubert syndrome 23
MONDO:0859221 Yoon-Bellen neurodevelopmental syndrome


## Undiagnosed

In [31]:
undiagnosed_phenotypes = [
    "HP:0001263", # Global developmental delay,
    "HP:0001250", # Hearing impairment,
    "HP:0002020", # Gastroesophageal reflux,
    "HP:0033454", # Tube feeding,
    "HP:0000097", # Focal segmental glomerulosclerosis,
    "HP:0001276", # Hypertonia,
    "HP:0012469", # Infantile spasms,
    "HP:0002521", # Hypsarrhythmia,
    "HP:0100704", # Cerebral visual impairment,
    "HP:0006970", # Periventricular leukomalacia,
    "HP:0002059" # Cerebral atrophy
]

In [32]:
response = phenotypes_to_disease(undiagnosed_phenotypes)
print_results(response)

200
MONDO:0032710 developmental and epileptic encephalopathy, 72
MONDO:0033371 developmental and epileptic encephalopathy, 62
MONDO:0030046 neurodevelopmental disorder and structural brain anomalies with or without seizures and spasticity
MONDO:0014947 developmental and epileptic encephalopathy, 46
MONDO:0013277 developmental and epileptic encephalopathy, 5
MONDO:0032768 developmental and epileptic encephalopathy, 76
MONDO:0014942 developmental and epileptic encephalopathy, 45
MONDO:0013351 infantile cerebral and cerebellar atrophy with postnatal progressive microcephaly
MONDO:0015205 isolated lissencephaly type 1 without known genetic defects
MONDO:0859313 neurodevelopmental disorder with speech impairment and with or without seizures


## Rett syndrome, GABA-A related disorder

In [33]:
rett_phenotypes = [
    "HP:0002013", # Vomiting,
    "HP:0001252", # Hypotonia,
    "HP:0001263", # Global developmental delay,
    "HP:0100543", # Cognitive impairment,
    "HP:0000750", # Delayed speech and language development,
    "HP:0001250", # Seizure,
    "HP:0033044" # Motor regression 
]

In [34]:
response = phenotypes_to_disease(rett_phenotypes)
print_results(response)

200
MONDO:0010451 intellectual disability, X-linked 41
MONDO:0859086 intellectual developmental disorder, X-linked 110
MONDO:0013697 intellectual disability, autosomal recessive 29
MONDO:0014524 intellectual disability, autosomal recessive 47
MONDO:0054861 intellectual disability, autosomal recessive 63
MONDO:0010450 intellectual disability, X-linked 89
MONDO:0010413 intellectual disability, X-linked 95
MONDO:0030911 intellectual disability, autosomal dominant 46
MONDO:0013705 intellectual disability, autosomal recessive 19
MONDO:0011710 specific language impairment 1
